# Setup

In [1]:
!pip install tensorflow==2.3.2
!pip install kerassurgeon
!pip install keras==2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 KB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 71.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
   

In [2]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import kerassurgeon
from kerassurgeon.operations import delete_channels

In [3]:
print(tf.__version__)
print(keras.__version__)
print(kerassurgeon.__version__)

2.3.2
2.4.0
0.2.0


In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Train model without pruning



Load MNIST dataset

In [5]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


Normalize the input image so that each pixel value is between 0 and 1.

In [6]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Define the model architecture


In [7]:
input = keras.layers.Input(shape=(28, 28))
reshape = keras.layers.Reshape(target_shape=(28, 28, 1))(input)
conv_1= keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu', name="conv_1")(reshape)
conv_2 = keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu', name="conv_2")(conv_1)
maxpool = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv_2)
flatten = keras.layers.Flatten()(maxpool)
dense = keras.layers.Dense(10)(flatten)

model =tf.keras.Model(inputs=[input], outputs=[dense])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 26, 26, 12)        120       
_________________________________________________________________
conv_2 (Conv2D)              (None, 24, 24, 12)        1308      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1728)              0         
_________________________________________________________________
dense (Dense)                (None, 10)               

Train the digit classification model

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=2,
  validation_split=0.1,
)

Epoch 1/2
1688/1688 [==============================] - 42s 25ms/step - loss: 0.2205 - accuracy: 0.9347 - val_loss: 0.0821 - val_accuracy: 0.9790
Epoch 2/2
1688/1688 [==============================] - 44s 26ms/step - loss: 0.0807 - accuracy: 0.9763 - val_loss: 0.0628 - val_accuracy: 0.9828


Evaluate baseline test accuracy (loss and accuracy)

In [9]:
model.evaluate(
    test_images, test_labels, verbose=0)

[0.0636981874704361, 0.9793000221252441]

In [10]:
tf.keras.models.save_model(model, "/content/gdrive/MyDrive/Curso-Jetson/models/mnist_model_2conv.h5", include_optimizer=False)

# Pruning weights L1 norm

Save layers name and weights

In [11]:
w_list = []
layer_name = []

for i in range(len(model.layers)): 
    if ('keras.layers.Conv3D' in model.layers[i]._keras_api_names[0]) or ('keras.layers.Conv2D' in model.layers[i]._keras_api_names[0]):
        print(model.layers[i]._keras_api_names[0])
        w = model.layers[i].get_weights()[0]
        w_list.append(w)
        layer_name.append(model.layers[i].name)


keras.layers.Conv2D
keras.layers.Conv2D


Calculate L1-norm for every filter

In [12]:
values_total = []

for i in range(len(w_list)):
    weight = w_list[i]
    weight_dict = {}

    num_filters = len(weight[0, 0, 0, :])

    for j in range(num_filters):
        w_s = np.sum(abs(weight[:, :, :, j]))
        weight_dict[j] = w_s

    values = np.fromiter(weight_dict.values(), dtype=float)
    values_total = np.append(values_total, values)

    weights_dict_sort = sorted(weight_dict.items(), key=lambda k: k[1])
    print("L1 norm conv layer {}\n".format(i + 1), weights_dict_sort)

L1 norm conv layer 1
 [(10, 0.87408555), (1, 1.2171497), (3, 1.3152757), (9, 1.6418724), (5, 1.6475912), (0, 1.6859784), (4, 1.6980257), (11, 1.8032663), (7, 1.8263495), (6, 1.9128357), (2, 1.929857), (8, 1.9926293)]
L1 norm conv layer 2
 [(5, 11.148841), (4, 11.300719), (9, 11.561567), (0, 11.581647), (8, 12.323187), (2, 12.93309), (6, 12.950718), (1, 16.540323), (7, 17.61103), (11, 18.019434), (10, 20.10907), (3, 20.629942)]


Clone model without pruning to preserve original model

In [13]:
model_filters= keras.models.clone_model(model)
model_filters.set_weights(model.get_weights())

Remove lower filters values according to a small percentile

In [14]:
values_total = np.sort(values_total)
valor_percentile = 50
percentile = np.percentile(values_total, valor_percentile)

counter = 0
deleted = 0

for i in range(len(model_filters.layers)): 
    if model_filters.layers[i].name in layer_name:
      weight = w_list[counter]
      num_filters = len(weight[0, 0, 0, :])

      for j in reversed(range(num_filters)):
          w_s = np.sum(abs(weight[:, :, :, j]))
          if w_s < percentile and model_filters.layers[i].output_shape[-1] > 1:
                model_filters = delete_channels(model_filters, model_filters.layers[i], np.array([j]))
                deleted = deleted + 1

      counter = counter + 1
      
print("Percentile value " + str(percentile))
print("{} Filters removed".format(deleted))

Deleting 1/12 channels from layer: conv_1
Deleting 1/11 channels from layer: conv_1
Deleting 1/10 channels from layer: conv_1
Deleting 1/9 channels from layer: conv_1
Deleting 1/8 channels from layer: conv_1
Deleting 1/7 channels from layer: conv_1
Deleting 1/6 channels from layer: conv_1
Deleting 1/5 channels from layer: conv_1
Deleting 1/4 channels from layer: conv_1
Deleting 1/3 channels from layer: conv_1
Deleting 1/2 channels from layer: conv_1
Percentile value 6.5707350969314575
11 Filters removed


Pruned model

In [15]:
model_filters.summary()

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 26, 26, 1)         10        
_________________________________________________________________
conv_2 (Conv2D)              (None, 24, 24, 12)        120       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1728)              0         
_________________________________________________________________
dense (Dense)                (None, 10)              

Test accuracy pruned model

In [16]:
model_filters.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_filters.evaluate(
    test_images, test_labels, verbose=0)

[1.843855619430542, 0.3375000059604645]

In [17]:
tf.keras.models.save_model(model_filters, "/content/gdrive/MyDrive/Curso-Jetson/models/mnist_model_2conv_pruned.h5", include_optimizer=False)

# Fine-tune pruned model

In [18]:
model_filters.fit(
  train_images,
  train_labels,
  epochs=2,
  validation_split=0.1,
)

Epoch 1/2
1688/1688 [==============================] - 30s 18ms/step - loss: 0.2377 - accuracy: 0.9323 - val_loss: 0.1111 - val_accuracy: 0.9698
Epoch 2/2
1688/1688 [==============================] - 26s 15ms/step - loss: 0.1071 - accuracy: 0.9675 - val_loss: 0.0843 - val_accuracy: 0.9765


In [19]:
model_filters.evaluate(
    test_images, test_labels, verbose=0)

[0.08673708140850067, 0.9725000262260437]

In [20]:
tf.keras.models.save_model(model_filters, "/content/gdrive/MyDrive/Curso-Jetson/models/mnist_model_2conv_ft.h5", include_optimizer=False)